# Build a Simple LLM Application with LCEL
---
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

## LCEL stands for LangChain Expression Language.

 - It is a declarative, functional programming interface used in LangChain to build and compose chains (pipelines) in a more readable and modular way — without writing custom Python classes or logic.

- Think of LCEL like Unix pipes (|) for LLM workflows.
It allows you to build logic like:

chain = prompt | llm | output_parser

- Each part is a component:

prompt: formats your input

llm: runs the language model

output_parser: parses the model output

All of these are LCEL components, and the | operator connects them into a pipeline.

- LCEL is like A universal language inside LangChain for connecting components like prompts, models, output parsers, retrievers, etc., using a clean, pipe-like syntax (|).

In [ ]:


import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")


# Open Source models--Llama3,Gemma2,mistral--Groq

groq_api_key=os.getenv("GROQ_API_KEY")

* Groq is a hardware and software company that designs ultra-fast AI inference chips and systems. It's known for developing high-performance AI accelerators that are used to run large language models (LLMs) and other machine learning workloads at extremely low latency

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="deepseek-r1-distill-llama-70b",groq_api_key=groq_api_key)
model

In [ ]:
# Whenever we chat with LLMs we should specify which message is provided by the human being 
# and which message is the instruction  -> system message

from langchain_core.messages import HumanMessage,SystemMessage


messages=[
    SystemMessage(content="Translate the following from English to japanese"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [ ]:
result
# AIMessage means that the ouput is a response from the LLM model

In [ ]:
print(result.content)

### ✅ To Extract Only the Final Answer:

1. import re
This imports Python's Regular Expressions module — re is used for finding and manipulating patterns in text (like removing tags).

---
2. re.sub(pattern, replacement, string, flags=...)
This function searches for all parts of a string that match a pattern and replaces them with something else.

pattern: What to find → we give it a regex pattern to match <think>...</think>

replacement: What to replace it with → we use an empty string "" (means "remove it")

string: The input string → result.content (LLM’s full output)

flags=re.DOTALL: This tells the regex to treat newlines (\n) as normal characters, so it can match multi-line <think>...</think> blocks

---

3. r"<think>.*?</think>"
This is the regex pattern:

<think>: matches the literal starting tag

.*?: matches anything inside (lazy, so it stops at the first </think>)

</think>: matches the closing tag

So the pattern matches everything between <think> and </think>, including the tags themselves.

---
4. .strip()
After replacing the <think>...</think> part with an empty string, we use .strip() to remove any extra spaces or newlines at the start or end.

In [ ]:
import re

cleaned_output = re.sub(r"<think>.*?</think>", "", result.content, flags=re.DOTALL).strip()
print(cleaned_output)


In [ ]:
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser used to for displaying only the response coming from LLM model
parser=StrOutputParser()
parser.invoke(result)

## Using LCEL
- To chain the components

In [ ]:
# Chaning means one after the other(eg: chain of thoufgt models)

chain=model|parser
chain.invoke(messages)

In [ ]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [ ]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [ ]:
result.to_messages()

In [ ]:
##Chaining together components with LCEL

chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})